In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [6]:
# get training and validation(=testing) data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/mantagaori/Desktop/comp473/COMP_473_Project/TrainSet2-new', 
    image_size = (48,48), # our original dataset is 48 pixels by 48 pixels
    batch_size = 20 # pick 12 images and trin until all dataset is used <= repeat this for each epoche
)


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/Users/mantagaori/Desktop/comp473/COMP_473_Project/TestSet2-new', 
    image_size = (48,48), 
    batch_size = 20
)

Found 3465 files belonging to 7 classes.
Found 1440 files belonging to 7 classes.


### No Pre-Processing

In [8]:
model = tf.keras.Sequential([

    tf.keras.layers.Conv2D( 32, (3,3), padding='same' , activation='relu', input_shape=(48,48,3)), 
    # pooling layer (Downsampling)
    # downsize 2,2 pixels to 1,1
    tf.keras.layers.MaxPooling2D((2,2)),

    # Flatten-Dense
    tf.keras.layers.Flatten(), 
    ## 1st layer
    tf.keras.layers.Dense(64, activation="relu"), # number inside Dense is the number of nodes. usually use 2^n
    tf.keras.layers.Dropout(0.2), # helps prevent overfittng
    ## second layer. we must have 7 final nodes because we have 7 classes.
    tf.keras.layers.Dense(7, activation="softmax") # softmax: compress resulting number between 0~1, used in category problems. If you add up the probability of each class we get 1. 
])



# compile and fit our model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam" , metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=100)

Epoch 1/100
174/174 [==============================] - 17s 85ms/step - loss: 18.7310 - accuracy: 0.2234 - val_loss: 1.7503 - val_accuracy: 0.2500
Epoch 2/100
174/174 [==============================] - 13s 72ms/step - loss: 1.6567 - accuracy: 0.3541 - val_loss: 1.5375 - val_accuracy: 0.4646
Epoch 3/100
174/174 [==============================] - 12s 70ms/step - loss: 1.5257 - accuracy: 0.4087 - val_loss: 1.4781 - val_accuracy: 0.4660
Epoch 4/100
174/174 [==============================] - 13s 73ms/step - loss: 1.5073 - accuracy: 0.4035 - val_loss: 1.4931 - val_accuracy: 0.4674
Epoch 5/100
174/174 [==============================] - 12s 70ms/step - loss: 1.5488 - accuracy: 0.3887 - val_loss: 1.4338 - val_accuracy: 0.4611
Epoch 6/100
174/174 [==============================] - 14s 82ms/step - loss: 1.4807 - accuracy: 0.4043 - val_loss: 1.4423 - val_accuracy: 0.4667
Epoch 7/100
174/174 [==============================] - 12s 68ms/step - loss: 1.4508 - accuracy: 0.4107 - val_loss: 1.3845 - val_a

##### Summary of No Pre-Processing model

In [9]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 flatten_3 (Flatten)         (None, 18432)             0         
                                                                 
 dense_6 (Dense)             (None, 64)                1179712   
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 7)                 455       
                                                      